In [ ]:
!pip install happytransformer
!pip install errant

In [ ]:
from happytransformer import HappyTextToText
from happytransformer import TTSettings
from happytransformer import TTTrainArgs
import pandas as pd
import torch
from datasets import load_dataset
import csv
from datasets import load_dataset, load_metric

In [ ]:
model_path = '/content/drive/MyDrive/Google Colab/Advanced NLP/GEC_C4_model/'
T5 = HappyTextToText(load_path=model_path)

04/04/2022 10:23:51 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [ ]:
# 4384 samples in BEA-dev
testFile = '/content/drive/MyDrive/Google Colab/Advanced NLP Project/Datasets/Test_Sets/BEA_dev.csv' 
test_df = pd.read_csv(testFile)
test_df['input'] = test_df['input'].str.lstrip()

In [ ]:
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=256)
incorrect_list, correct_list = [], []
cnt = 0
for i in range(test_df.shape[0]):
  incorrect = test_df.iloc[i]['input']
  incorrect_list.append(incorrect)
  result = T5.generate_text('grammar: ' + incorrect, args=beam_settings)
  correct_list.append(result.text)
  cnt+=1
  if cnt%100==0:
    print(cnt)

In [ ]:
incorrect_ref_list, correct_ref_list = [], []
for i in range(test_df.shape[0]):
  incorrect_ref_list.append(test_df.iloc[i]['input'])
  correct_ref_list.append(test_df.iloc[i]['target'])

In [ ]:
def write_txt_file(filePath, str_list):
  with open(filePath, 'w') as f:
    for item in str_list:
      f.write("%s\n" % item)

In [ ]:
incorrect_ref_file = '/content/incorrect_ref.txt'
write_txt_file(incorrect_ref_file, incorrect_ref_list)

correct_ref_file = '/content/correct_ref.txt'
write_txt_file(correct_ref_file, correct_ref_list)

In [ ]:
!errant_parallel -orig '/content/incorrect_ref.txt' -cor '/content/correct_ref.txt' -out '/content/ref.m2'

Loading resources...
Processing parallel files...


In [ ]:
correct_pred_file = '/content/correct_pred.txt'
write_txt_file(correct_pred_file, correct_list)

In [ ]:
!errant_parallel -orig '/content/incorrect_ref.txt' -cor '/content/correct_pred.txt' -out '/content/pred.m2'

Loading resources...
Processing parallel files...


In [ ]:
!errant_compare -hyp '/content/pred.m2' -ref '/content/ref.m2'


=========== Span-Based Correction ============
TP	FP	FN	Prec	Rec	F0.5
1132	1644	6449	0.4078	0.1493	0.3029

